You are given a dataset containing sales data for an e-commerce website. The dataset (task\sales_data.csv) has the following columns:

Date: Date of the sale.
Product: Name of the product sold.
Category: Category to which the product belongs.
Quantity: Number of units sold.
Price: Price per unit.

In [6]:
import pandas as pd

# 1. Load the data
df = pd.read_csv('sales_data.csv', parse_dates=['Date'])

# 2. Aggregate stats per Category
agg_category = df.groupby('Category').agg(
    total_quantity=('Quantity', 'sum'),
    average_price=('Price', 'mean'),
    max_quantity_per_transaction=('Quantity', 'max')
).reset_index()

print("Aggregate stats by category:")
print(agg_category.to_string(index=False))


# 3. Top‐selling product per category
prod_totals = (
    df.groupby(['Category', 'Product'])['Quantity']
      .sum()
      .reset_index()
)
top_products = (
    prod_totals.sort_values(['Category', 'Quantity'], ascending=[True, False])
               .groupby('Category')
               .first()
               .reset_index()
               .rename(columns={
                   'Product': 'top_selling_product',
                   'Quantity': 'total_quantity_sold'
               })
)

print("\nTop‐selling product by category:")
print(top_products.to_string(index=False))


# 4. Date with highest total sales
df['Sales'] = df['Quantity'] * df['Price']
daily_sales = df.groupby('Date')['Sales'].sum().reset_index()
top_day = daily_sales.loc[daily_sales['Sales'].idxmax()]

print(f"\nDate with highest total sales:")
print(f"  {top_day['Date'].date()} — Total Sales = ${top_day['Sales']:.2f}")


Aggregate stats by category:
   Category  total_quantity  average_price  max_quantity_per_transaction
   Clothing             157      31.176471                            15
Electronics             183     276.764706                            15
       Home             144      55.000000                            14

Top‐selling product by category:
   Category top_selling_product  total_quantity_sold
   Clothing               Jeans                   15
Electronics            Smart TV                   15
       Home     Pressure Cooker                   14

Date with highest total sales:
  2023-01-07 — Total Sales = $15150.00


In [8]:
import pandas as pd

# Load the data
df = pd.read_csv('customer_orders.csv')  # adjust path if needed

# 1. Customers with at least 20 orders
order_counts = (
    df.groupby('CustomerID')
      .size()
      .reset_index(name='order_count')
)
customers_20_plus = order_counts[order_counts['order_count'] >= 20]
print("Customers with ≥20 orders:")
print(customers_20_plus)

# 2. Customers with average price per unit > $120
avg_price = (
    df.groupby('CustomerID')['Price']
      .mean()
      .reset_index(name='avg_price_per_unit')
)
customers_avg_gt_120 = avg_price[avg_price['avg_price_per_unit'] > 120]
print("\nCustomers with average unit price > $120:")
print(customers_avg_gt_120)

# 3. Product totals, filtering out total quantity < 5
df['TotalPrice'] = df['Quantity'] * df['Price']
product_stats = (
    df.groupby('Product')
      .agg(
          total_quantity=('Quantity', 'sum'),
          total_price=('TotalPrice', 'sum')
      )
      .reset_index()
)
products_qty_5_plus = product_stats[product_stats['total_quantity'] >= 5]
print("\nProducts with total quantity ≥5:")
print(products_qty_5_plus)


Customers with ≥20 orders:
   CustomerID  order_count
0         101           21
1         102           21
2         103           20
3         104           20

Customers with average unit price > $120:
   CustomerID  avg_price_per_unit
1         102          138.095238
3         104          169.750000

Products with total quantity ≥5:
             Product  total_quantity  total_price
5        Cargo Pants               6          180
15       Dress Shirt               5          125
19      Formal Shirt               6          210
30        Smartphone               5         2000
32       Sport Shoes               5          200
35        Sunglasses               5           75
41  Wireless Earbuds               6          720


In [14]:
import sqlite3
import pandas as pd
import openpyxl

# 1. Load population table via SQL
conn = sqlite3.connect('population.db')
# assuming table is named 'population' with columns at least ['PersonID','State','Salary']
pop_df = pd.read_sql_query("SELECT * FROM population", conn)
conn.close()

# 2. Load salary bands from Excel
bands_df = pd.read_excel('population_salary_analysis.xlsx')

# 3. Assign each person to a band
#    Assume bands_df has columns ['BandName', 'MinSalary', 'MaxSalary']
#    We'll do a merge‑like assignment via pandas interval mapping
bands_df['interval'] = pd.IntervalIndex.from_arrays(
    bands_df['Lower Bound'], bands_df['Upper Bound'], closed='both'
)
# Create a categorical Series mapping each salary into the right interval
pop_df['Band'] = pd.cut(
    pop_df['Salary'],
    bins=bands_df['interval'],
    labels=bands_df['BandName']
)

# 4A. Overall statistics per Band
total_pop = len(pop_df)

overall = pop_df.groupby('Band')['Salary'].agg(
    count='size',
    average_salary='mean',
    median_salary='median'
).reset_index()
overall['percentage'] = overall['count'] / total_pop * 100

print("=== Overall by Salary Band ===")
print(overall[['Band', 'percentage', 'average_salary', 'median_salary', 'count']])

# 4B. Same measures, broken out by State and Band
by_state = (
    pop_df
    .groupby(['State', 'Band'])['Salary']
    .agg(
        count='size',
        average_salary='mean',
        median_salary='median'
    )
    .reset_index()
)

# Compute total population per State for percentage calculation
state_totals = pop_df.groupby('State').size().reset_index(name='state_total')

by_state = by_state.merge(state_totals, on='State')
by_state['percentage'] = by_state['count'] / by_state['state_total'] * 100

print("\n=== By State and Salary Band ===")
print(by_state[['State', 'Band', 'percentage', 'average_salary', 'median_salary', 'count']])


KeyError: 'Lower Bound'